In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]
output_dir = output_path / 'RTP/access'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
transit_jobs_access = pd.read_csv(output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', 
                         separator=' ', 
                         columns=['parcelid','emptot_p']).to_pandas()


# jobs access in equity geographies
equity_geogs = summary_config['equity_geogs_comb']

## Jobs Accessible within 45 Minutes of Transit

In [4]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']

    df_access_equity_geogs = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='1')].copy()
    df = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='0')].copy()

    df_access_equity_geogs['geography'] = df_access_equity_geogs['geography_group']
    df['geography'] = "NOT in " + df['geography_group']

    df_access_equity_geogs = pd.concat([df_access_equity_geogs, df], ignore_index=True)
    df_access_equity_geogs['geography_group'] = 'Equity Geography'

    return df_access, df_access_equity_geogs


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [5]:
def job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs'] = df['Jobs within 45-minute Transit Commute'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df


In [6]:
df = job_access_geog(df_access_t,'CountyName')

f_name = str(incr)+'_transit_jobs_45min_county_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"402,066.6",12.7%
Kitsap,"19,136.5",0.6%
Outside Region,0.0,0.0%
Pierce,"68,855.8",2.2%
Snohomish,"113,283.3",3.6%
Region,"251,916.9",8.0%


In [7]:
df_rgc = job_access_geog(df_access_t,'rgc_binary')
df = job_access_geog(df_access_t,'GrowthCenterName')

df = pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

f_name = str(incr)+'_transit_jobs_45min_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"251,916.9",8.0%
Not in RGC,"184,004.5",5.8%
In RGC,"551,829.5",17.5%
Auburn,"202,535.3",6.4%
Bellevue,"755,282.4",23.9%
Bothell Canyon Park,"100,649.8",3.2%
Bremerton,"67,653.6",2.1%
Burien,"351,148.5",11.1%
Everett,"210,525.5",6.7%


In [8]:
df = job_access_geog(df_access_t,'rg_proposed')

f_name = str(incr)+'_transit_jobs_45min_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"251,916.9",8.0%
CitiesTowns,"25,655.0",0.8%
Core,"167,540.4",5.3%
HCT,"123,710.1",3.9%
Metro,"519,664.0",16.4%
UU,"27,429.5",0.9%


In [9]:
df = job_access_geog(df_access_equity_t,'Equity Geography')

f_name = str(incr)+'_transit_jobs_45min_equity.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [10]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [11]:
df = bp_job_access_geog(df_access_bp,'CountyName')

f_name = str(incr)+'_walk_bike_jobs_county_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,"34,702.7","129,887.5",1.1%,4.1%
Kitsap,"2,678.8","14,069.2",0.1%,0.4%
Outside Region,0.8,15.8,0.0%,0.0%
Pierce,"8,912.7","38,235.2",0.3%,1.2%
Snohomish,"7,397.2","36,878.7",0.2%,1.2%
Region,"21,947.1","85,234.3",0.7%,2.7%


In [12]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

df = pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

f_name = str(incr)+'_walk_bike_jobs_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"21,947.1","85,234.3",0.7%,2.7%
Not in RGC,"4,629.9","49,699.4",0.1%,1.6%
In RGC,"98,423.0","242,162.4",3.1%,7.7%
Auburn,"24,238.3","61,079.5",0.8%,1.9%
Bellevue,"107,708.2","181,749.4",3.4%,5.7%
Bothell Canyon Park,"10,928.3","36,499.8",0.3%,1.2%
Bremerton,"21,851.2","47,392.5",0.7%,1.5%
Burien,"10,215.3","23,017.9",0.3%,0.7%
Everett,"56,056.8","96,508.9",1.8%,3.1%


In [13]:
df = bp_job_access_geog(df_access_bp,'rg_proposed')

f_name = str(incr)+'_walk_bike_jobs_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"21,947.1","85,234.3",0.7%,2.7%
CitiesTowns,"1,347.1","12,948.6",0.0%,0.4%
Core,"8,258.8","48,762.2",0.3%,1.5%
HCT,"2,712.1","26,253.2",0.1%,0.8%
Metro,"54,751.1","189,054.9",1.7%,6.0%
UU,654.0,"10,431.6",0.0%,0.3%


In [14]:
df = bp_job_access_geog(df_access_equity_bp,'Equity Geography')

f_name = str(incr)+'_walk_bike_jobs_equity.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,


## Intersection Density

In [15]:
buffered_parcels = pl.read_csv(output_path / 'landuse/buffered_parcels.txt', 
                               separator=' ',
                               columns=['parcelid','nodes3_2','nodes4_2','hh_p'])


async_engine = create_engine('sqlite:///' + summary_config['sc_run_path'] + '/inputs/db/' + config['db_name'])

list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed'] + equity_geogs
parcel_geog = pl.read_database(
    query= f"SELECT {', '.join(list_cols)} FROM " + "parcel_" + config["base_year"] + "_geography",
    connection=async_engine.connect()
)

In [16]:
df_intersection = buffered_parcels.join(parcel_geog, left_on='parcelid', right_on='ParcelID').to_pandas()

# Total intersections within 1/2 mile buffer
df_intersection['intersections_wt'] = (df_intersection['nodes3_2'] + df_intersection['nodes4_2']) * df_intersection['hh_p']

In [17]:
def intersection_density(geog):
    df = df_intersection.groupby(geog)[['intersections_wt', 'hh_p']].sum().reset_index()
    df['Intersections'] = df['intersections_wt']/df['hh_p']
    
    return df[[geog] + ['Intersections']]

In [18]:
intersection_density('CountyName')

,CountyName,Intersections
0,King,168.4
1,Kitsap,54.5
2,Outside Region,2.2
3,Pierce,90.9
4,Snohomish,81.6


In [19]:
intersection_density('GrowthCenterName')

,GrowthCenterName,Intersections
0,Auburn,194.4
1,Bellevue,272.5
2,Bothell Canyon Park,71.5
3,Bremerton,166.8
4,Burien,177.1
5,Everett,158.5
6,Federal Way,128.7
7,Greater Downtown Kirkland,164.2
8,Issaquah,110.4
9,Kent,209.0


In [20]:
intersection_density('rg_proposed')

,rg_proposed,Intersections
0,CitiesTowns,63.4
1,Core,111.4
2,HCT,86.6
3,Metro,213.8
4,UU,50.8


In [21]:
df_intersection_equity = df_intersection.loc[df_intersection['disability_geog_vs_reg_total']>=0].copy()

# get total intersections by equity geography
df1 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['intersections_wt']).sum().reset_index()
df1.columns = ['Equity Group', 'intersections_wt']

# get total households by equity geography
df2 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['hh_p']).sum().reset_index()
df2.columns = ['Equity Group', 'hh_p']

df_in = df1.merge(df2, on='Equity Group')
df_in['Inside Equity Geography'] = df_in['intersections_wt']/df_in['hh_p']

# outside equity geographies
df1_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['intersections_wt']).sum().reset_index()
df1_out.columns = ['Equity Group', 'intersections_wt']
df2_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['hh_p']).sum().reset_index()
df2_out.columns = ['Equity Group', 'hh_p']
df_out = df1_out.merge(df2_out, on='Equity Group')
df_out['Outside Equity Geography'] = df_out['intersections_wt']/df_out['hh_p']

df = df_in[['Equity Group','Inside Equity Geography']].merge(
    df_out[['Equity Group','Outside Equity Geography']], on='Equity Group')

df

,Equity Group,Inside Equity Geography,Outside Equity Geography
0,youth_geog_vs_reg_total,78.6,163.5
1,elderly_geog_vs_reg_total,113.7,140.3
2,english_geog_vs_reg_total,136.8,122.3
3,racial_geog_vs_reg_total,152.8,104.8
4,racial_geog_vs_50_percent,156.4,113.9
5,poverty_geog_vs_reg_total,140.2,118.4
6,poverty_geog_vs_50_percent,212.9,125.5
7,disability_geog_vs_reg_total,129.2,126.9
